<a href="https://colab.research.google.com/github/HarshaMupparaju/IPARC/blob/main/iparc_challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
from google.colab import drive
drive.mount('/content/gdrive')

%cd gdrive/My Drive/ML project

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
[Errno 2] No such file or directory: 'gdrive/My Drive/ML project'
/content/gdrive/My Drive/ML project


In [34]:
import os
from collections import Counter
from torch.utils.data import Dataset, DataLoader
import torch
import numpy as np
import matplotlib.pyplot as plt
import json
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from PIL import Image

In [35]:
from scipy.ndimage import binary_dilation, binary_erosion
import json
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from skimage.morphology import disk

# Square
SE1_3x3 = np.ones((3, 3), dtype=np.int32)
SE1_5x5 = np.ones((5, 5), dtype=np.int32)

# Disk
SE2_3x3 = np.array(disk(1), dtype=np.int32)
SE2_5x5 = np.array(disk(2), dtype=np.int32)

# Cross
SE3_3x3 = np.zeros((3, 3), dtype=np.int32)
SE3_3x3[(0, 1, 2), (0, 1, 2)] = 1
SE3_3x3[(0, 1, 2), (2, 1, 0)] = 1
SE3_5x5 = np.zeros((5, 5), dtype=np.int32)
SE3_5x5[(0, 1, 2, 3, 4), (0, 1, 2, 3, 4)] = 1
SE3_5x5[(0, 1, 2, 3, 4), (4, 3, 2, 1, 0)] = 1

# Plus
SE4_3x3 = np.zeros((3, 3), dtype=np.int32)
SE4_3x3[1, :] = 1
SE4_3x3[:, 1] = 1
SE4_5x5 = np.zeros((5, 5), dtype=np.int32)
SE4_5x5[2, :] = 1
SE4_5x5[:, 2] = 1

# Rhombus
SE5_3x3 = np.array(disk(1), dtype=np.int32)
SE5_3x3[1, 1] = 0
SE5_5x5 = np.array(disk(2), dtype=np.int32)
SE5_5x5[(1, 2, 2, 2, 3), (2, 1, 2, 3, 3)] = 0

# Square (Empty)
SE7_3x3 = np.ones((3, 3), dtype=np.int32)
SE7_3x3[1, 1] = 0
SE7_5x5 = np.ones((5, 5), dtype=np.int32)
SE7_5x5[1:4, 1:4] = 0

# Line to right
SE8_3x3 = np.zeros((3, 3), dtype=np.int32)
SE8_3x3[:, 2] = 1
SE8_5x5 = np.zeros((5, 5), dtype=np.int32)
SE8_5x5[:, 4] = 1

# Line to left
SE9_3x3 = np.zeros((3, 3), dtype=np.int32)
SE9_3x3[:, 0] = 1
SE9_5x5 = np.zeros((5, 5), dtype=np.int32)
SE9_5x5[:, 0] = 1

# Line to Top
SE10_3x3 = np.zeros((3, 3), dtype=np.int32)
SE10_3x3[0, :] = 1
SE10_5x5 = np.zeros((5, 5), dtype=np.int32)
SE10_5x5[0, :] = 1

# Line to Bottom
SE11_3x3 = np.zeros((3, 3), dtype=np.int32)
SE11_3x3[-1, :] = 1
SE11_5x5 = np.zeros((5, 5), dtype=np.int32)
SE11_5x5[-1, :] = 1

list_se_3x3 = [SE3_3x3, SE4_3x3, SE5_3x3, SE7_3x3, SE8_3x3, SE9_3x3, SE10_3x3, SE11_3x3]


def get_img(img, se_list):
    list_se = ['SE1', 'SE2', 'SE3', 'SE4', 'SE5', 'SE6', 'SE7', 'SE8']
    for se in se_list:
        list_se_idx = list_se.index(se)
        img = binary_dilation(img, list_se_3x3[list_se_idx])

    for se in se_list:
        list_se_idx = list_se.index(se)
        img = binary_erosion(img, list_se_3x3[list_se_idx])

    return img

In [36]:
path = r"/content/gdrive/MyDrive/ML project/IPARC_ChallengeV2/Dataset/CatA_Simple/"
class_lst = [
    "SE8",
    "SE4",
    "SE6",
    "SE7",
    "SE5",
    "SE2",
    "SE3",
    "SE1",
]
class_mapping = {class_name: index for index, class_name in enumerate(class_lst)}


def labels_to_indices(labels, class_mapping):
    indices = [class_mapping[label] for label in labels]
    return torch.tensor(indices, dtype=torch.long)


def perform_ops(img, op, se):
    return img


class IPARC(Dataset):
    def __init__(self, path, device="cpu"):
        self.path = path
        self.device = device

    def __getitem__(self, idx):
        file_list = os.listdir(self.path)

        txt_list = [x for x in file_list if x.endswith(".txt")]
        json_list = [x for x in file_list if x.endswith(".json")]

        with open(self.path + json_list[idx]) as f:
            data = json.load(f)

        with open(self.path + txt_list[idx], "r") as f:
            label_list = [line.strip() for line in f.readlines()]

        label_list = label_list[:4]
        label_list = [x.split(" ")[1] for x in label_list]
        label_list = list(set([class_lst.index(x) for x in label_list]))
        
        arr = []
        out = []

        for i in data:
            arr.append(i["input"])
            out.append(i["output"])
            
        label = np.zeros((8))

        arr = torch.tensor(np.array(arr)).float()
        out = torch.tensor(np.array(out)).float()
        labels = torch.tensor(np.array(label_list)).float()
        input_image = torch.cat([arr, out], dim= 0)
        
        for i in label_list:
            label[int(i)] = 1.

        return input_image, torch.tensor(label).float()

    def __len__(self):
        return len(os.listdir(self.path)) // 2


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
data = IPARC(path, device)
train, val, test = torch.utils.data.random_split(
    data, [int(len(data) * 0.8), int(len(data) * 0.1), int(len(data) * 0.1)]
)
train_loader = DataLoader(train, batch_size=10, shuffle=True)
test_loader = DataLoader(test, batch_size=2, shuffle=True)
val_loader = DataLoader(val, batch_size=2, shuffle=True)

print("Dataset loaded.")

Dataset loaded.


In [37]:
class SimpleCNN(nn.Module):
    def __init__(self, num_classes=8):
        super(SimpleCNN, self).__init__()

        self.conv1 = nn.Sequential(
            nn.Conv2d(8, 4, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.fc2 = nn.Sequential(nn.Linear(196, 16), nn.ReLU())
        self.output = nn.Linear(16, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = x.view(x.size(0), -1)
        x = self.fc2(x)
        x = self.output(x)
        return x


num_classes = len(class_lst)
model = SimpleCNN(num_classes)
model.cuda()

print("Model loaded.")

Model loaded.


In [38]:
model(torch.zeros(1, 8, 15, 15).cuda())

tensor([[-0.1567,  0.1833,  0.1251, -0.1413,  0.1073,  0.1056,  0.1984, -0.1630]],
       device='cuda:0', grad_fn=<AddmmBackward0>)

In [39]:
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay= 1e-4)

class LossIPARC(nn.Module):
    def __init__(self):
        super(LossIPARC, self).__init__()
        self.ce = nn.CrossEntropyLoss().cuda()
        # self.mse = nn.MSELoss()

    def forward(self, input_, target):
        
        loss = self.ce(input_, target) + self.mse(input_, target)
        
        input_numpy, target_numpy = input_.clone(), target.clone()
        
        for i, j in zip(input_, target):
            print(i)

        return loss

criterion_1 = LossIPARC()
criterion = nn.CrossEntropyLoss().to(device)

In [40]:
def train_and_evaluate(model, train_loader, test_loader, optimizer, criterion, device, epochs, log_interval):
    history = {
        "train_loss": [],
        "test_loss": []
    }

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        train_loss = 0.0
        for batch_idx, (input_image, labels) in enumerate(train_loader):
            input_image, labels = input_image.to(device), labels.to(device)

            optimizer.zero_grad()
            output = model(input_image)

            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            train_loss += loss.item()

            if (batch_idx + 1) % log_interval == 0:
                print("Epoch [{}/{}], Step [{}/{}], Train Loss: {:.4f}".format(
                    epoch + 1, epochs, batch_idx + 1, len(train_loader), running_loss / log_interval))
                running_loss = 0.0

        history["train_loss"].append(train_loss / len(train_loader))

        model.eval()
        test_loss = 0.0

        with torch.no_grad():
            for input_image, labels in test_loader:
                input_image, labels = input_image.to(device), labels.to(device)

                output = model(input_image)

                loss = criterion(output, labels)
                test_loss += loss.item()

        history["test_loss"].append(test_loss / len(test_loader))
        print("Epoch [{}/{}], Test Loss: {:.4f}".format(epoch + 1, epochs, test_loss / len(test_loader)))

    return history

In [41]:
epochs, log_interval = 100, 100
print("Starting training.")
history = train_and_evaluate(model, train_loader, test_loader, optimizer, criterion, device, epochs, log_interval)

print(history)

Starting training.
Epoch [1/100], Test Loss: 7.0820
Epoch [2/100], Test Loss: 7.0329
Epoch [3/100], Test Loss: 6.9898
Epoch [4/100], Test Loss: 6.9752
Epoch [5/100], Test Loss: 6.9586
Epoch [6/100], Test Loss: 6.9370
Epoch [7/100], Test Loss: 6.9208
Epoch [8/100], Test Loss: 6.9391
Epoch [9/100], Test Loss: 6.9330
Epoch [10/100], Test Loss: 6.9183
Epoch [11/100], Test Loss: 6.9138
Epoch [12/100], Test Loss: 6.9074
Epoch [13/100], Test Loss: 6.9050
Epoch [14/100], Test Loss: 6.8999
Epoch [15/100], Test Loss: 6.9135
Epoch [16/100], Test Loss: 6.9117
Epoch [17/100], Test Loss: 6.9152
Epoch [18/100], Test Loss: 6.9436
Epoch [19/100], Test Loss: 6.9634
Epoch [20/100], Test Loss: 6.9658
Epoch [21/100], Test Loss: 6.9968
Epoch [22/100], Test Loss: 7.0562
Epoch [23/100], Test Loss: 7.0688
Epoch [24/100], Test Loss: 7.0918
Epoch [25/100], Test Loss: 7.1650
Epoch [26/100], Test Loss: 7.1781
Epoch [27/100], Test Loss: 7.2276
Epoch [28/100], Test Loss: 7.2619
Epoch [29/100], Test Loss: 7.3515
Epoc

In [52]:
# Test Model

input_img, label = val[6]
output = model(input_img.unsqueeze(0).cuda()).squeeze(0)

# criterion_1(output, label)
output = list(output.detach().cpu().numpy())
label = list(label.detach().cpu().numpy())
out_fns, label_fns = [], []

for i, x in enumerate(output):
    if x > 1:
        out_fns.append(class_lst[i])
for i, x in enumerate(label):
    if x == 1:
        label_fns.append(class_lst[i])
        
out_fns, label_fns

(['SE8', 'SE5', 'SE1'], ['SE6', 'SE5', 'SE2', 'SE1'])